In [8]:
import pandas as pd

In [17]:
to_predict_mens = pd.read_csv('to_predict_mens.csv')


In [18]:
to_predict_mens = pd.read_csv('to_predict_mens.csv')
teams = pd.read_csv('../Data/MTeams.csv')
teams1 = teams.rename(columns = {"TeamID": "Team1", "TeamName":"Team1_Name"})
to_predict_mens = to_predict_mens.merge(teams1, how="left", on="Team1")
teams2 = teams.rename(columns = {"TeamID": "Team2", "TeamName":"Team2_Name"})
to_predict_mens = to_predict_mens.merge(teams2, how="left", on="Team2")
kaggle_df = to_predict_mens[to_predict_mens.type == "Historical"]

In [19]:
kaggle_df = to_predict_mens[to_predict_mens.type == "Historical"]

In [20]:
first_round_odds_data = pd.read_csv('../data/first_round_odds_data.csv')

first_round_odds_data["Date"] = pd.to_datetime(first_round_odds_data["Date"], format='%b %d, %Y')

# Extract the year
first_round_odds_data['year'] = first_round_odds_data["Date"].dt.year

kaggle_df["year"] = kaggle_df["ID"].apply(lambda x: x[0:4])
kaggle_df["year"] = kaggle_df["year"].astype(int)

first_round_odds_data1 = first_round_odds_data.rename(columns={"kaggle_team": "Team1_Name",
                                                               "odds": "odds_team1"})
kaggle_df = kaggle_df.merge(first_round_odds_data1, how="left", on=["year", "Team1_Name"])

first_round_odds_data2 = first_round_odds_data.rename(columns={"kaggle_team": "Team2_Name",
                                                               "odds": "odds_team2"
                                                               })
kaggle_df = kaggle_df.merge(first_round_odds_data2, how="left", on=["year", "Team2_Name"])

import numpy as np
kaggle_df["final_odds"] = np.where(kaggle_df.odds_team1.isna(),
                                    kaggle_df.odds_team2 * -1, 
                                    kaggle_df.odds_team1 )


final_df = kaggle_df[(~kaggle_df.final_odds.isna())
                     & (~kaggle_df.Total_x.isna())                     
                     ].copy()     



/var/folders/2g/465yxy_x4g786jx2llr5xqh40000gn/T/ipykernel_2721/2816333077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_df["year"] = kaggle_df["ID"].apply(lambda x: x[0:4])
/var/folders/2g/465yxy_x4g786jx2llr5xqh40000gn/T/ipykernel_2721/2816333077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_df["year"] = kaggle_df["year"].astype(int)


In [24]:
# kaggle_df[(kaggle_df.odds_team1.isna())
#           & (kaggle_df.odds_team2.isna())          
#           ][["Team1_Name", "Team2_Name", "year"]]

                               

### Model

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss


In [29]:
#final_df_full = final_df.copy()
# final_df = final_df_full.copy()

In [30]:
def split_first_round(df):

   first_round_df = df[

      ( (df.t1_Seed == 1) & (df.t2_Seed == 16) ) |
      ( (df.t1_Seed == 2) & (df.t2_Seed == 15) ) |
      ( (df.t1_Seed == 3) & (df.t2_Seed == 14) ) |
      ( (df.t1_Seed == 4) & (df.t2_Seed == 13) ) |
      ( (df.t1_Seed == 5) & (df.t2_Seed == 12) ) |
      ( (df.t1_Seed == 6) & (df.t2_Seed == 11) ) |
      ( (df.t1_Seed == 7) & (df.t2_Seed == 10) ) |
      ( (df.t1_Seed == 8) & (df.t2_Seed == 9) ) |
      ( (df.t1_Seed == 9) & (df.t2_Seed == 8) ) |
      ( (df.t1_Seed == 10) & (df.t2_Seed == 7) ) |
      ( (df.t1_Seed == 11) & (df.t2_Seed == 6) ) |
      ( (df.t1_Seed == 12) & (df.t2_Seed == 5) ) |
      ( (df.t1_Seed == 13) & (df.t2_Seed == 4) ) |
      ( (df.t1_Seed == 14) & (df.t2_Seed == 3) ) |
      ( (df.t1_Seed == 15) & (df.t2_Seed == 2) ) |
      ( (df.t1_Seed == 16) & (df.t2_Seed == 1) ) 
   ].copy()

   other_rounds_df = df[

        ~(
      ( (df.t1_Seed == 1) & (df.t2_Seed == 16) ) |
      ( (df.t1_Seed == 2) & (df.t2_Seed == 15) ) |
      ( (df.t1_Seed == 3) & (df.t2_Seed == 14) ) |
      ( (df.t1_Seed == 4) & (df.t2_Seed == 13) ) |
      ( (df.t1_Seed == 5) & (df.t2_Seed == 12) ) |
      ( (df.t1_Seed == 6) & (df.t2_Seed == 11) ) |
      ( (df.t1_Seed == 7) & (df.t2_Seed == 10) ) |
      ( (df.t1_Seed == 8) & (df.t2_Seed == 9) ) |
      ( (df.t1_Seed == 9) & (df.t2_Seed == 8) ) |
      ( (df.t1_Seed == 10) & (df.t2_Seed == 7) ) |
      ( (df.t1_Seed == 11) & (df.t2_Seed == 6) ) |
      ( (df.t1_Seed == 12) & (df.t2_Seed == 5) ) |
      ( (df.t1_Seed == 13) & (df.t2_Seed == 4) ) |
      ( (df.t1_Seed == 14) & (df.t2_Seed == 3) ) |
      ( (df.t1_Seed == 15) & (df.t2_Seed == 2) ) |
      ( (df.t1_Seed == 16) & (df.t2_Seed == 1) )
        ) 
   ].copy()

   return first_round_df, other_rounds_df

In [35]:
full_df = final_df.copy()

In [36]:
first_round_df, other_rounds_df = split_first_round(final_df)


In [37]:
final_df = first_round_df.copy()

In [72]:
# final_df = final_df_full.copy()

train = final_df[final_df.Season < 2017].copy()
test = final_df[final_df.Season >= 2017].copy()

first_round_features = [
            'seed_diff', 
              't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank',
                'final_odds', 
                'Total_x']


first_round_features = [
            'seed_diff', 
              't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank',
                # 'final_odds', 
                # 'Total_x'
                ]


other_round_features = [
            'seed_diff', 
              't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank']

features = first_round_features

target = 'Outcome'

train[target] = train[target].astype(int)

train[features] = StandardScaler().fit_transform(train[features])

In [73]:
from sklearn.ensemble import RandomForestClassifier

In [74]:
def get_best_params_from_grid_search(train, features, target, model, parameters, scoring, verbose=10):
    gs = GridSearchCV(estimator=model, param_grid=parameters, scoring=scoring, verbose=verbose)
    gs.fit(train[features], train[target])
    return gs.best_params_, gs

model = LogisticRegression(random_state=23)
parameters = {'C':[.00001, .0001, .001, .01, .1, 1, 100, 1000]}

# model = RandomForestClassifier(random_state=23)
# parameters = {'min_samples_leaf':[2, 3, 4, 5, 10],
#               'max_depth':[2, 3, 4, 5, 10],}

best_params, gs = get_best_params_from_grid_search(train, features, target, model, parameters, 
                                               scoring='neg_brier_score', verbose=0)

In [75]:
# first round
gs.best_score_, gs.best_params_

(-0.16746171439609286, {'C': 0.01})

In [76]:
# other round
gs.best_score_, gs.best_params_

(-0.16746171439609286, {'C': 0.01})

In [77]:
def rolling_season_cv(first_round_model, other_round_model, train_input, first_round_features, other_round_features, label='Outcome', cv_start=2007):
    ''' returns estimate for model performance using shifted validation'''
    
    scores = []

    for n, season in enumerate(train_input.Season.unique()):
        
        if season >= cv_start:

            train = train_input[train_input.Season < season]
            test = train_input[train_input.Season == season]

            train_first_round_df, train_other_rounds_df = split_first_round(train)
            test_first_round_df, test_other_rounds_df = split_first_round(test)

            # first round
            first_round_X_train = train_first_round_df[first_round_features]
            first_round_X_test = test_first_round_df[first_round_features]
            first_round_y_train = train_first_round_df[label]
            first_round_y_test = test_first_round_df[label]
            
            first_round_model.fit(first_round_X_train[first_round_features], first_round_y_train)
            first_round_y_prob = first_round_model.predict_proba(first_round_X_test[first_round_features])[:,1]

            # other round
            other_round_X_train = train_other_rounds_df[other_round_features]
            other_round_X_test = test_other_rounds_df[other_round_features]
            other_round_y_train = train_other_rounds_df[label]
            other_round_y_test = test_other_rounds_df[label]
            
            other_round_model.fit(other_round_X_train[other_round_features], other_round_y_train)
            other_round_y_prob = other_round_model.predict_proba(other_round_X_test[other_round_features])[:,1]

            y_test = pd.concat([first_round_y_test, other_round_y_test], axis=0)
            y_prob = np.concatenate((first_round_y_prob, other_round_y_prob), axis=0)
            
            loss = brier_score_loss(y_test, y_prob)
            scores.append((season, loss))

    validation_df = pd.DataFrame(scores, columns = ['season', 'score']).sort_values(by = 'score')
    avg_validation_score = validation_df.score.mean()
    return avg_validation_score


In [78]:
def rolling_season_cv(model, train_input, features, label='Outcome', cv_start=2007):
    ''' returns estimate for model performance using shifted validation'''
    
    scores = []

    for n, season in enumerate(train_input.Season.unique()):
        
        if season >= cv_start:

            train = train_input[train_input.Season < season]
            test = train_input[train_input.Season == season]

            X_train = train[features]
            X_test = test[features]
            y_train = train[label]
            y_test = test[label]

            model.fit(X_train[features], y_train)
            y_prob = model.predict_proba(X_test[features])
            loss = brier_score_loss(y_test, y_prob[:,1])
            scores.append((season, loss))

    validation_df = pd.DataFrame(scores, columns = ['season', 'score']).sort_values(by = 'score')
    avg_validation_score = validation_df.score.mean()
    return avg_validation_score


In [79]:
first_round_model = LogisticRegression(C=.01)

rolling_season_cv(first_round_model, train,                   
                  first_round_features)

0.17084577628955183

In [71]:
first_round_model = LogisticRegression(C=.1)

rolling_season_cv(first_round_model, train,                   
                  first_round_features)

0.17083748120747583

In [45]:
first_round_model = LogisticRegression(C=1)

rolling_season_cv(first_round_model, train,                   
                  first_round_features)

0.1679768416553255

In [448]:
# model = RandomForestClassifier(**{'max_depth': 5, 
#                                   'min_samples_leaf': 4,
#                                   'n_estimators': 1000})
# rolling_season_cv(model, train, features)

In [453]:
other_round_model = LogisticRegression(C=.1)


In [454]:
rolling_season_cv(first_round_model, other_round_model, train,                   
                  first_round_features, other_round_features)

0.1794061331648122

In [286]:
model = LogisticRegression(C=.01)
rolling_season_cv(model, train, features)

0.18230801009918612

In [ ]:
model = LogisticRegression(C=.01)
rolling_season_cv(model, train, features)

0.18023711399612496

In [ ]:
model = LogisticRegression(C=.01)
rolling_season_cv(model, train, features)

0.18029456175758835

In [ ]:
0.18029456175758835 - 0.18023711399612496

5.74477614633806e-05

In [ ]:
0.17084577628955183 - 0.16991629637393696

0.0009294799156148725